## הטמעות

בדוגמה הקודמת שלנו, עבדנו עם וקטורי bag-of-words בעלי ממד גבוה באורך `vocab_size`, והמרנו באופן מפורש וקטורי ייצוג מיקום בעלי ממד נמוך לייצוג דל מסוג one-hot. ייצוג זה אינו יעיל מבחינת זיכרון. בנוסף, כל מילה מטופלת באופן עצמאי, כך שווקטורים מקודדים ב-one-hot אינם מבטאים דמיון סמנטי בין מילים.

ביחידה זו, נמשיך לחקור את מערך הנתונים **News AG**. כדי להתחיל, נטען את הנתונים ונקבל כמה הגדרות מהיחידה הקודמת.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### מהו embedding?

הרעיון של **embedding** הוא לייצג מילים באמצעות וקטורים צפופים בממדים נמוכים יותר, שמשקפים את המשמעות הסמנטית של המילה. בהמשך נדון כיצד לבנות embeddings משמעותיים, אבל לעת עתה נחשוב על embeddings כדרך להפחית את הממדיות של וקטור מילה.

לכן, שכבת embedding מקבלת מילה כקלט ומפיקה וקטור פלט בגודל `embedding_size` שנבחר. במובן מסוים, זה דומה מאוד לשכבת `Dense`, אבל במקום לקבל וקטור מקודד one-hot כקלט, היא יכולה לקבל מספר שמייצג מילה.

על ידי שימוש בשכבת embedding כשכבה הראשונה ברשת שלנו, אנחנו יכולים לעבור ממודל bag-of-words למודל **embedding bag**, שבו תחילה אנו ממירים כל מילה בטקסט שלנו ל-embedding המתאים לה, ואז מחשבים פונקציית צבירה כלשהי על כל ה-embeddings, כמו `sum`, `average` או `max`.

![תמונה המציגה מסווג embedding עבור חמישה רצפי מילים.](../../../../../translated_images/he/embedding-classifier-example.b77f021a7ee67eee.webp)

רשת הנוירונים המסווגת שלנו מורכבת מהשכבות הבאות:

* שכבת `TextVectorization`, שמקבלת מחרוזת כקלט ומפיקה טנזור של מספרי טוקנים. נגדיר גודל אוצר מילים סביר `vocab_size`, ונתעלם ממילים שמשתמשים בהן פחות. צורת הקלט תהיה 1, וצורת הפלט תהיה $n$, מכיוון שנקבל $n$ טוקנים כתוצאה, שכל אחד מהם מכיל מספרים בין 0 ל-`vocab_size`.
* שכבת `Embedding`, שמקבלת $n$ מספרים ומצמצמת כל מספר לוקטור צפוף באורך נתון (100 בדוגמה שלנו). כך, טנזור קלט בצורת $n$ יומר לטנזור בצורת $n\times 100$.
* שכבת צבירה, שמחשבת את הממוצע של הטנזור הזה לאורך הציר הראשון, כלומר היא תחשב את הממוצע של כל $n$ הטנזורים הקלטיים שמייצגים מילים שונות. כדי ליישם שכבה זו, נשתמש בשכבת `Lambda`, ונעביר לתוכה את הפונקציה לחישוב הממוצע. הפלט יהיה בצורת 100, והוא יהיה הייצוג המספרי של כל רצף הקלט.
* מסווג ליניארי סופי מסוג `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


בפלט `summary`, בעמודת **output shape**, הממד הראשון של הטנזור `None` מתייחס לגודל המיני-אצווה, והממד השני מתייחס לאורך רצף הטוקנים. כל רצפי הטוקנים במיני-אצווה הם באורכים שונים. נדון כיצד להתמודד עם זה בסעיף הבא.

עכשיו בואו נתחיל לאמן את הרשת:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


**שימו לב** שאנחנו בונים וקטורייזר על בסיס תת-קבוצה של הנתונים. הדבר נעשה כדי להאיץ את התהליך, וזה עשוי להוביל למצב שבו לא כל הטוקנים מהטקסט שלנו נמצאים באוצר המילים. במקרה כזה, הטוקנים הללו ייעלמו, מה שעשוי לגרום לדיוק מעט נמוך יותר. עם זאת, בחיים האמיתיים תת-קבוצה של טקסט לעיתים קרובות מספקת הערכה טובה של אוצר המילים.


### התמודדות עם גדלים משתנים של רצפי משתנים

בואו נבין איך מתבצע אימון במיניבאצ'ים. בדוגמה למעלה, למערך הקלט יש מימד 1, ואנחנו משתמשים במיניבאצ'ים באורך 128, כך שהגודל בפועל של המערך הוא $128 \times 1$. עם זאת, מספר הטוקנים בכל משפט הוא שונה. אם ניישם את השכבה `TextVectorization` על קלט יחיד, מספר הטוקנים המוחזר יהיה שונה, בהתאם לאופן שבו הטקסט עובר תהליך טוקניזציה:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


עם זאת, כאשר אנו מיישמים את הוקטורייזר על מספר רצפים, עליו לייצר טנזור בצורת מלבן, ולכן הוא ממלא אלמנטים לא בשימוש עם הטוקן PAD (שבמקרה שלנו הוא אפס):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

כאן אנו יכולים לראות את ההטמעות:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

**הערה**: כדי למזער את כמות הריפוד, במקרים מסוימים יש היגיון למיין את כל הרצפים במאגר הנתונים לפי סדר עולה של אורך (או, ליתר דיוק, מספר הטוקנים). כך יובטח שכל מיני-אצווה תכיל רצפים באורך דומה.


## הטמעות סמנטיות: Word2Vec

בדוגמה הקודמת שלנו, שכבת ההטמעה למדה למפות מילים לייצוגים וקטוריים, אך לייצוגים הללו לא הייתה משמעות סמנטית. יהיה נחמד ללמוד ייצוג וקטורי כך שמילים דומות או מילים נרדפות יתאימו לוקטורים הקרובים זה לזה במונחים של מרחק וקטורי מסוים (לדוגמה, מרחק אוקלידי).

כדי לעשות זאת, עלינו לאמן מראש את מודל ההטמעה שלנו על אוסף גדול של טקסט באמצעות טכניקה כמו [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). הטכניקה מבוססת על שתי ארכיטקטורות עיקריות המשמשות ליצירת ייצוג מבוזר של מילים:

 - **Continuous bag-of-words** (CBoW), שבה אנו מאמנים את המודל לנבא מילה מתוך ההקשר הסובב אותה. בהינתן הנגרם $(W_{-2},W_{-1},W_0,W_1,W_2)$, מטרת המודל היא לנבא את $W_0$ מתוך $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** הוא ההפך מ-CBoW. המודל משתמש בחלון ההקשר של המילים הסובבות כדי לנבא את המילה הנוכחית.

CBoW מהיר יותר, בעוד ש-Skip-Gram איטי יותר, אך הוא עושה עבודה טובה יותר בייצוג מילים נדירות.

![תמונה המציגה את האלגוריתמים CBoW ו-Skip-Gram להמרת מילים לוקטורים.](../../../../../translated_images/he/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

כדי להתנסות בהטמעת Word2Vec שאומנה מראש על מאגר הנתונים של Google News, ניתן להשתמש בספריית **gensim**. להלן נאתר את המילים הדומות ביותר ל'neural'.

> **Note:** כשאתם יוצרים לראשונה וקטורי מילים, הורדתם עשויה לקחת זמן!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


אנחנו יכולים גם לחלץ את הטמעת הווקטור מהמילה, לשימוש באימון מודל הסיווג. לטמעה יש 300 רכיבים, אך כאן אנו מציגים רק את 20 הרכיבים הראשונים של הווקטור לצורך הבהרה:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

הדבר הנהדר בהטמעות סמנטיות הוא שניתן לתמרן את קידוד הווקטור בהתבסס על סמנטיקה. לדוגמה, אנו יכולים לבקש למצוא מילה שהייצוג הווקטורי שלה קרוב ככל האפשר למילים *מלך* ו-*אישה*, ורחוק ככל האפשר מהמילה *גבר*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

דוגמה לעיל משתמשת בכמה קסמים פנימיים של GenSym, אך ההיגיון הבסיסי למעשה די פשוט. דבר מעניין לגבי הטמעות הוא שניתן לבצע פעולות וקטור רגילות על וקטורי הטמעות, וזה ישקף פעולות על **משמעויות** של מילים. הדוגמה לעיל יכולה להתבטא במונחים של פעולות וקטור: אנו מחשבים את הווקטור המתאים ל-**KING-MAN+WOMAN** (פעולות `+` ו-`-` מתבצעות על ייצוגי וקטור של מילים מתאימות), ואז מוצאים את המילה הקרובה ביותר במילון לווקטור הזה:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **הערה**: היינו צריכים להוסיף מקדמים קטנים לווקטורים של *man* ו-*woman* - נסו להסיר אותם ולראות מה קורה.

כדי למצוא את הווקטור הקרוב ביותר, אנו משתמשים במנגנון של TensorFlow כדי לחשב וקטור של מרחקים בין הווקטור שלנו לבין כל הווקטורים באוצר המילים, ואז מוצאים את האינדקס של המילה המינימלית באמצעות `argmin`.


בעוד ש-Word2Vec נראה כמו דרך מצוינת לבטא סמנטיקה של מילים, יש לו חסרונות רבים, כולל הבאים:

* גם מודלי CBoW וגם skip-gram הם **הטמעות ניבוי**, והם מתחשבים רק בהקשר המקומי. Word2Vec אינו מנצל את ההקשר הגלובלי.
* Word2Vec אינו מתחשב ב**מורפולוגיה** של מילים, כלומר העובדה שמשמעות המילה יכולה להיות תלויה בחלקים שונים של המילה, כמו השורש.

**FastText** מנסה להתגבר על המגבלה השנייה, ובונה על Word2Vec על ידי למידת ייצוגי וקטור עבור כל מילה וה-n-grams של תווים שנמצאים בתוך כל מילה. הערכים של הייצוגים ממוצעים לאחר מכן לווקטור אחד בכל שלב אימון. למרות שזה מוסיף הרבה חישוב נוסף לשלב ההכנה, זה מאפשר להטמעות המילים לקודד מידע תת-מילתי.

שיטה נוספת, **GloVe**, משתמשת בגישה שונה להטמעות מילים, המבוססת על פירוק מטריצת ההקשר של המילים. ראשית, היא בונה מטריצה גדולה שסופרת את מספר ההופעות של מילים בהקשרים שונים, ולאחר מכן היא מנסה לייצג את המטריצה הזו בממדים נמוכים באופן שממזער את אובדן השחזור.

ספריית gensim תומכת בהטמעות המילים הללו, ואתם יכולים להתנסות בהן על ידי שינוי קוד טעינת המודל שמופיע למעלה.


## שימוש באמבדינגים מוכנים מראש ב-Keras

ניתן לשנות את הדוגמה לעיל כדי למלא מראש את המטריצה בשכבת האמבדינג שלנו עם אמבדינגים סמנטיים, כמו Word2Vec. סביר להניח שהמילונים של האמבדינג המוכן מראש ושל קורפוס הטקסט לא יתאימו זה לזה, ולכן עלינו לבחור אחד מהם. כאן נבחן את שתי האפשרויות האפשריות: שימוש במילון של הטוקנייזר, ושימוש במילון של אמבדינגים מ-Word2Vec.

### שימוש במילון של הטוקנייזר

כאשר משתמשים במילון של הטוקנייזר, חלק מהמילים במילון יהיו בעלות אמבדינגים תואמים מ-Word2Vec, וחלקן יהיו חסרות. בהתחשב בכך שגודל המילון שלנו הוא `vocab_size`, ואורך וקטור האמבדינג של Word2Vec הוא `embed_size`, שכבת האמבדינג תיוצג על ידי מטריצת משקל בצורת `vocab_size`$\times$`embed_size`. נמלא את המטריצה הזו על ידי מעבר דרך המילון:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


עבור מילים שאינן קיימות באוצר המילים של Word2Vec, אנחנו יכולים להשאיר אותן כאפסים, או ליצור וקטור אקראי.

כעת נוכל להגדיר שכבת הטמעה עם משקלים שהוכנו מראש:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: שימו לב שהגדרנו `trainable=False` בעת יצירת ה-`Embedding`, מה שאומר שאנחנו לא מאמנים מחדש את שכבת ה-Embedding. זה עשוי לגרום לדיוק להיות מעט נמוך יותר, אך זה מאיץ את תהליך האימון.

### שימוש באוצר מילים של Embedding

בעיה אחת בגישה הקודמת היא שאוצר המילים המשמש ב-TextVectorization וב-Embedding שונה. כדי להתגבר על הבעיה הזו, ניתן להשתמש באחת מהפתרונות הבאים:
* לאמן מחדש את מודל ה-Word2Vec על אוצר המילים שלנו.
* לטעון את מערך הנתונים שלנו עם אוצר המילים ממודל ה-Word2Vec המאומן מראש. ניתן להגדיר את אוצר המילים המשמש לטעינת מערך הנתונים במהלך הטעינה.

הגישה השנייה נראית פשוטה יותר, אז בואו ניישם אותה. קודם כל, ניצור שכבת `TextVectorization` עם אוצר מילים מוגדר, שנלקח מה-Word2Vec embeddings:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

ספריית ההטמעות של gensim מכילה פונקציה נוחה, `get_keras_embeddings`, אשר תיצור באופן אוטומטי את שכבת ההטמעות המתאימה של Keras עבורך.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


אחת הסיבות לכך שאנו לא רואים דיוק גבוה יותר היא משום שחלק מהמילים ממאגר הנתונים שלנו חסרות באוצר המילים המוקדם של GloVe, ולכן הן למעשה מתעלמות. כדי להתגבר על כך, אנו יכולים לאמן את ההטבעות שלנו בהתבסס על מאגר הנתונים שלנו.


## הטמעות הקשריות

אחת המגבלות המרכזיות של ייצוגי הטמעות מסורתיים שהוכנו מראש, כמו Word2Vec, היא העובדה שאמנם הם יכולים ללכוד חלק מהמשמעות של מילה, אך הם אינם מסוגלים להבחין בין משמעויות שונות. מגבלה זו יכולה לגרום לבעיות במודלים שמבוססים עליהם.

לדוגמה, למילה 'play' יש משמעויות שונות בשני המשפטים הבאים:
- הלכתי ל**הצגה** בתיאטרון.
- ג'ון רוצה **לשחק** עם חבריו.

ההטמעות שהוכנו מראש עליהן דיברנו מייצגות את שתי המשמעויות של המילה 'play' באותה הטמעה. כדי להתגבר על מגבלה זו, עלינו לבנות הטמעות המבוססות על **מודל שפה**, שמאומן על מאגר טקסטים גדול ו*יודע* כיצד מילים יכולות להשתלב בהקשרים שונים. דיון בהטמעות הקשריות חורג מתחום המדריך הזה, אך נחזור אליהן כשנדבר על מודלי שפה ביחידה הבאה.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
